In [47]:
import os
import pandas as pd
import numpy as np

import keras
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

In [48]:
_image_dir = '../datasets/celeba/img_align_celeba_png/'
_img_width = 178
_img_height = 218
_batch_size = 16
_num_epochs = 20

In [49]:
img_df = pd.read_csv('../datasets/celeba/attributes/attr_celeba.csv').replace({-1:0})[['filename', 'Eyeglasses', 'Smiling', 'Wearing_Lipstick', 'No_Beard', 'Young']]
partition_df = pd.read_csv('../datasets/celeba/attributes/list_eval_partition.csv')
img_df = img_df.merge(partition_df, on='filename')
img_df.loc[:, 'filename'] = img_df.loc[:, 'filename'].apply(lambda x: os.path.join(_image_dir, x))
img_df.head()

,filename,Eyeglasses,Smiling,Wearing_Lipstick,No_Beard,Young,partition
0,../datasets/celeba/img_align_celeba_png/000001...,0,1,1,1,1,0
1,../datasets/celeba/img_align_celeba_png/000002...,0,1,0,1,1,0
2,../datasets/celeba/img_align_celeba_png/000003...,0,0,0,1,1,0
3,../datasets/celeba/img_align_celeba_png/000004...,0,0,1,1,1,0
4,../datasets/celeba/img_align_celeba_png/000005...,0,0,1,1,1,0


In [50]:
train_df = img_df.loc[img_df.loc[:, 'partition'] == 0, ['filename', 'Eyeglasses']]
valid_df = img_df.loc[img_df.loc[:, 'partition'] == 1, ['filename', 'Eyeglasses']]
test_df = img_df.loc[img_df.loc[:, 'partition'] == 2, ['filename', 'Eyeglasses',]]

In [52]:
train_datagen =  ImageDataGenerator(
#     rotation_range=30,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     brightness_range=[0.4,1.5],
    preprocessing_function=preprocess_input,
    rescale=1./255,
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    batch_size=_batch_size,
    x_col='filename', 
    y_col='Eyeglasses',
    class_mode='raw',
    target_size=(_img_width, _img_height),
    validate_filenames=False)

Found 162770 non-validated image filenames.


In [53]:
valid_datagen =  ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1./255
)

valid_generator = valid_datagen.flow_from_dataframe(
    valid_df,
    batch_size=_batch_size,
    x_col='filename', 
    y_col='Eyeglasses',
    class_mode='raw',
    target_size=(_img_width, _img_height),
    validate_filenames=False)

Found 19867 non-validated image filenames.


In [54]:
base_model = InceptionV3(weights='imagenet',
                        include_top=False,
                        input_shape=(_img_width, _img_height, 3))

print("number of layers:", len(base_model.layers))

for layer in model.layers[:52]:
    layer.trainable = False

87916544/87910968 [==============================] - 56s 1us/step
number of layers: 311


In [55]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(512, activation="relu")(x)
predictions = Dense(1, activation="sigmoid")(x)

In [56]:
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [57]:
checkpointer = ModelCheckpoint(filepath='inception_v3_eyeglasses.h5', verbose=1, save_best_only=True)

In [ ]:
hist = model.fit_generator(train_generator,
                            validation_data = valid_generator,
                            steps_per_epoch= len(train_df)//_batch_size,
                            validation_steps=len(valid_df) // _batch_size,
                            epochs= _num_epochs,
                            callbacks=[checkpointer],
                            verbose=1,
                           )

Epoch 1/20
10173/10173 [==============================] - 1329s 131ms/step - loss: 0.0746 - accuracy: 0.9804 - val_loss: 0.1871 - val_accuracy: 0.9744

Epoch 00001: val_loss improved from inf to 0.18707, saving model to inception_v3_eyeglasses.h5
Epoch 2/20
10173/10173 [==============================] - 1325s 130ms/step - loss: 0.0327 - accuracy: 0.9914 - val_loss: 0.0038 - val_accuracy: 0.9519

Epoch 00002: val_loss improved from 0.18707 to 0.00385, saving model to inception_v3_eyeglasses.h5
Epoch 3/20
10173/10173 [==============================] - 1327s 130ms/step - loss: 0.0337 - accuracy: 0.9908 - val_loss: 0.5980 - val_accuracy: 0.9874

Epoch 00003: val_loss did not improve from 0.00385
Epoch 4/20
10173/10173 [==============================] - 1326s 130ms/step - loss: 0.0270 - accuracy: 0.9935 - val_loss: 2.6612e-04 - val_accuracy: 0.9911

Epoch 00004: val_loss improved from 0.00385 to 0.00027, saving model to inception_v3_eyeglasses.h5
Epoch 5/20
10173/10173 [====================